# Process pavement widths

Based on methodology outlined in https://github.com/meliharvey/sidewalkwidths-nyc

- read roadside (pavement) features
- find centreline
- calculate width along centreline
- calculate some summary statistics
- output cleaned

In [ ]:
import glob
import json
import os

import centerline
import centerline.exceptions
import pandas
import geopandas
import requests
import shapely.wkt

from centerline.geometry import Centerline
from pandarallel import pandarallel
from shapely.geometry import LineString
from shapely.geometry import Point, MultiPoint, MultiLineString
from shapely.ops import linemerge, nearest_points
from tqdm.notebook import tqdm

In [ ]:
# set up big figures
from matplotlib.pyplot import rcParams
rcParams['figure.figsize'] = 10, 10

In [ ]:
# set up progress_apply
tqdm.pandas()

In [ ]:
CACHE_PATH = os.path.join('.', 'db-data')

In [ ]:
df = geopandas.read_file(os.path.join(CACHE_PATH, 'E07000026.gpkg'))

In [ ]:
df = df.cx[300000:300200, 528800:529000]
df.plot()

In [ ]:
def remove_short_lines(line):
    
    if line.type == 'MultiLineString':
        
        passing_lines = []
    
        for i, linestring in enumerate(line):
            
            other_lines = MultiLineString([x for j, x in enumerate(line) if j != i])
            
            p0 = Point(linestring.coords[0])
            p1 = Point(linestring.coords[-1])
            
            is_deadend = False
            
            if p0.disjoint(other_lines): is_deadend = True
            if p1.disjoint(other_lines): is_deadend = True
            
            if not is_deadend or linestring.length > 5:                
                passing_lines.append(linestring)
            
        return MultiLineString(passing_lines)
            
    if line.type == 'LineString':
        return line

In [ ]:
# df['centerlines'] = df.progress_apply(lambda row: Centerline(row.geometry), axis=1)
df['centerlines'] = df.geometry.progress_apply(Centerline)

In [ ]:
df.centerlines = df.centerlines.progress_apply(linemerge)

In [ ]:
df.centerlines = df.centerlines.progress_apply(remove_short_lines)

In [ ]:
df.centerlines = df.centerlines.progress_apply(lambda line: line.simplify(1, preserve_topology=True))

In [ ]:
def linestring_to_segments(linestring):
    return [
        LineString([linestring.coords[i], linestring.coords[i+1]]) 
        for i in range(len(linestring.coords) - 1)
    ]

def get_segments(line):
    if line.type == 'MultiLineString': 
        line_segments = []
        for linestring in line.geoms:            
            line_segments.extend(linestring_to_segments(linestring))
        return line_segments

    elif line.type == 'LineString':        
        return linestring_to_segments(line)
    else:
        return []

In [ ]:
df['segments'] = df['centerlines'].progress_apply(get_segments)

In [ ]:
def interpolate_by_distance(linestring, distance=1):
    count = round(linestring.length / distance) + 1
    
    if count == 1:
        # grab mid-point if it's a short line
        return [linestring.interpolate(linestring.length / 2)]
    else:
        # interpolate along the line
        return [linestring.interpolate(distance * i) for i in range(count)]

def interpolate(line):    
    if line.type == 'MultiLineString':        
        all_points = []
        
        for linestring in line:
            all_points.extend(interpolate_by_distance(linestring))
        
        return all_points
            
    if line.type == 'LineString':
        return interpolate_by_distance(line)
    
    
def polygon_to_multilinestring(polygon):
    return MultiLineString([polygon.exterior] + [line for line in polygon.interiors])
    

def get_avg_distances(row):
    avg_distances = []
    
    boundary = polygon_to_multilinestring(row.geometry)
    
    for segment in row.segments:        
        points = interpolate(segment)
        
        distances = []
        
        for point in points:
            p1, p2 = nearest_points(boundary, point)
            distances.append(p1.distance(p2))
            
        avg_distances.append(sum(distances) / len(distances))
        
    return avg_distances

In [ ]:
df['avg_distances'] = df.progress_apply(get_avg_distances, axis=1)

In [ ]:
dfc = df.set_geometry('centerlines')

In [ ]:
df.plot()

In [ ]:
dfc.plot()

In [ ]:
def explode_to_segments(df):
    data = {'geometry': [], 'width': []}

    for i, row in df.iterrows():

        for segment, distance in zip(row.segments, row.avg_distances):
            data['geometry'].append(segment.buffer(distance))
            data['width'].append(distance * 2)

    df_segments = pandas.DataFrame(data)
    df_segments = geopandas.GeoDataFrame(df_segments, crs=df.crs, geometry='geometry')
    return df_segments

df_segments = explode_to_segments(df)
df_segments.head(1)

In [ ]:
ax = df.plot(figsize=(15,15), color='white')
ax.set_facecolor('black')
df_segments.plot(ax=ax, column='width', cmap='Spectral', legend=True)

# Process all

In [ ]:
lads = geopandas.read_file(os.path.join(CACHE_PATH, "lads.gpkg"))

In [ ]:
def process_centreline(geom):
    try:
        line = Centerline(geom)
    except centerline.exceptions.TooFewRidgesError:
        line = Centerline(geom, interpolation_distance=0.1)
        
    line = remove_short_lines(linemerge(line))
    return line.simplify(1, preserve_topology=True)

def process_lad(lad_code):
    try:
        df = geopandas.read_file(os.path.join(CACHE_PATH, f'{lad_code}.gpkg'))
    except Exception as err:
        print(lad_code, err)
        return
    
    df['centerlines'] = df.geometry.progress_apply(process_centreline)
    df['segments'] = df.centerlines.progress_apply(get_segments)
    df['avg_distances'] = df.progress_apply(get_avg_distances, axis=1)
    df_segments = explode_to_segments(df)
    df_segments.to_file(os.path.join(CACHE_PATH, f'{lad_code}_segments.gpkg'), driver='GPKG')    

In [ ]:
for lad_code in tqdm(lads.lad_code):
    if lad_code == 'E06000001':
        print(lad_code)
        process_lad(lad_code)